# 认识Bedrock

![](images/bedrock_models.png)

In [1]:
# !pip install -r requirements.txt

### boto3创建 `bedrock` client

In [3]:
import json
import boto3
import base64

# 指定要使用的配置文件名称
profile_name = 'default'

# 创建 AWS 客户端，并指定配置文件名称
session = boto3.Session(profile_name=profile_name)


#Create the connection to Bedrock
bedrock = session.client(
    service_name='bedrock',
    region_name='us-west-2', 
)

bedrock_runtime = session.client(
    service_name='bedrock-runtime',
    region_name='us-west-2', 
    
)

### 列出可用模型版本

In [4]:
# List all foundation models
all_llms = [ model['modelId'] for model in bedrock.list_foundation_models()['modelSummaries']]
all_llms

['amazon.titan-tg1-large',
 'amazon.titan-embed-g1-text-02',
 'amazon.titan-text-lite-v1:0:4k',
 'amazon.titan-text-lite-v1',
 'amazon.titan-text-express-v1:0:8k',
 'amazon.titan-text-express-v1',
 'amazon.titan-embed-text-v1:2:8k',
 'amazon.titan-embed-text-v1',
 'amazon.titan-embed-text-v2:0',
 'amazon.titan-embed-image-v1:0',
 'amazon.titan-embed-image-v1',
 'amazon.titan-image-generator-v1:0',
 'amazon.titan-image-generator-v1',
 'amazon.titan-image-generator-v2:0',
 'stability.stable-diffusion-xl-v1:0',
 'stability.stable-diffusion-xl-v1',
 'stability.sd3-large-v1:0',
 'stability.stable-image-core-v1:0',
 'stability.stable-image-ultra-v1:0',
 'anthropic.claude-3-5-sonnet-20241022-v2:0',
 'anthropic.claude-3-5-haiku-20241022-v1:0',
 'anthropic.claude-instant-v1:2:100k',
 'anthropic.claude-instant-v1',
 'anthropic.claude-v2:0:18k',
 'anthropic.claude-v2:0:100k',
 'anthropic.claude-v2:1:18k',
 'anthropic.claude-v2:1:200k',
 'anthropic.claude-v2:1',
 'anthropic.claude-v2',
 'anthropic

### `Anthropic (Claude v3 (Opus))`

In [ ]:
# Claude v3 (Opus)有强大的逻辑推理与解题能力

In [5]:
prompt_data = """小圆的半径是3cm,大圆的半径是5cm,大圆与小圆的面积比是()。
A.3:5
B.9:25
C.25:9
D.5:3"""

body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1024,
            "messages": [
                 {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt_data
                        }
                    ]
                }
            ],
        }

body = json.dumps(body) # Encode body as JSON string

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

In [6]:
response = bedrock_runtime.invoke_model(body=body,
                                        modelId=model_id, 
                                        accept=accept, 
                                        contentType=contentType)

In [7]:
response_body = json.loads(response.get("body").read())
for output in response_body.get("content", []):
    print(output["text"])

要求解大圆与小圆的面积比,我们需要计算出大圆和小圆的面积,然后求比值。

已知:
小圆半径r1 = 3cm
大圆半径r2 = 5cm

圆的面积公式为:
A = πr^2

那么:
小圆面积A1 = π * (3cm)^2 = 9π (cm^2)
大圆面积A2 = π * (5cm)^2 = 25π (cm^2)

大圆与小圆的面积比 = A2 / A1 = (25π) / (9π) = 25/9

因此,正确答案是C.25:9


### With `Mistral (mixtral-8x7b)`

In [ ]:
# Mistral (mixtral-8x7b) 是社区模型中performance突出的模型，这里我们使用mixtral-8x7b

In [9]:
prompt_data = """写一个德语B1的学习大纲"""

body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1024,
            "messages": [
                 {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt_data
                        }
                    ]
                }
            ],
        }

body = json.dumps(body) # Encode body as JSON string

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body.encode('utf-8'), # Encode to bytes
                                        modelId=model_id, 
                                        accept=accept, 
                                        contentType=contentType)

response_body = json.loads(response.get('body').read().decode('utf-8'))

for output in response_body.get("content", []):
    print(output["text"])


很高兴您有意深入学习德语,以下是一份德语B1级别的学习大纲供您参考:

一、听力理解
- 能够理解常见话题的主要内容,如工作、学习、休闲等
- 能够理解简单说明、广播、公告等
- 能够大致了解德语电影、电视剧的主要内容

二、阅读理解
- 能够阅读并理解实用的短文、说明、简单的报纸文章等
- 能够阅读并理解常见话题的非小说文学作品,如简单小说、故事等
- 能够大致了解常见话题的书面说明、手册等

三、口语表达
- 能够用德语进行日常生活的基本交流
- 能够用德语介绍自己的经历、计划、理想等
- 能够比较流利地描述事物,解释观点并举例说明

四、书面表达
- 能够书写常见话题的简单短文、信件等
- 能够以连贯的段落书写个人经历、感受、梦想等
- 能够以基本的语法结构和连接词组织文章

五、语法知识
- 完整掌握德语名词、代词、形容词等词类的格变位
- 掌握常用的时态和语气,如现在时、完成时等
- 掌握常见的从句类型,比如定语从句、状语从句等

六、词汇量
- 积累约2500到3000个常用生活词汇
- 掌握约800到1000个常见词根和词缀
- 能够理解并运用常见的固定搭配和习语

以上是德语B1级别的一个基本框架,具体的学习内容和进度可根据个人情况做适当的调整。希望这份大纲能够给您带来一些启发和帮助。


In [10]:
for output in response_body.get("content", []):
    print(output["text"])

很高兴您有意深入学习德语,以下是一份德语B1级别的学习大纲供您参考:

一、听力理解
- 能够理解常见话题的主要内容,如工作、学习、休闲等
- 能够理解简单说明、广播、公告等
- 能够大致了解德语电影、电视剧的主要内容

二、阅读理解
- 能够阅读并理解实用的短文、说明、简单的报纸文章等
- 能够阅读并理解常见话题的非小说文学作品,如简单小说、故事等
- 能够大致了解常见话题的书面说明、手册等

三、口语表达
- 能够用德语进行日常生活的基本交流
- 能够用德语介绍自己的经历、计划、理想等
- 能够比较流利地描述事物,解释观点并举例说明

四、书面表达
- 能够书写常见话题的简单短文、信件等
- 能够以连贯的段落书写个人经历、感受、梦想等
- 能够以基本的语法结构和连接词组织文章

五、语法知识
- 完整掌握德语名词、代词、形容词等词类的格变位
- 掌握常用的时态和语气,如现在时、完成时等
- 掌握常见的从句类型,比如定语从句、状语从句等

六、词汇量
- 积累约2500到3000个常用生活词汇
- 掌握约800到1000个常见词根和词缀
- 能够理解并运用常见的固定搭配和习语

以上是德语B1级别的一个基本框架,具体的学习内容和进度可根据个人情况做适当的调整。希望这份大纲能够给您带来一些启发和帮助。


### 多模态图片输入 `Anthropic Claude v3 Haiku`

In [12]:
!pwd

/home/ec2-user/SageMaker/bedrock_ci_north


In [14]:
with open("images/math.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
    base64_string = encoded_string.decode('utf-8')

payload = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": base64_string
                        }
                    },
                    {
                        "type": "text",
                        "text": "As shown in the figure, in the rectangle quadrilateral ABCD, if E is a point on the edge AB, DAE folded along the crease DE to DA'E, Please independently assess the correctness of the following three statements and provide a complete analysis: \n1. The trajectory of the projection of point A' is F, AF is perpendicular to DE;\n2. AF intersects DE at O, When E varies, the trajectory of O is a circle with AD as the diameter;\n3. The trajectory of point A' is a circle with O as the center and AO as the radius;"
                    }
                ]
            }
        ]
    }
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body_bytes,
                                        modelId=model_id, 
                                        accept=accept, 
                                        contentType=contentType)


In [15]:
response_body = json.loads(response.get('body').read().decode('utf-8'))
print(response_body.get('content')[0].get('text'))

Let me assess the correctness of each statement and provide a complete analysis:

1. The trajectory of the projection of point A' is F, AF is perpendicular to DE.
This statement is correct. When the fold occurs along the crease DE, point A' moves along a straight line that is perpendicular to the crease DE. This straight line is the projection of point A' onto the line DE, and the point where this projection intersects DE is denoted as F. Therefore, the trajectory of the projection of point A' is the point F, and AF is indeed perpendicular to DE.

2. AF intersects DE at O, When E varies, the trajectory of O is a circle with AD as the diameter.
This statement is also correct. As E moves along the line segment AB, the point of intersection O between AF and DE also moves. However, the locus of points O forms a circle with AD as its diameter. This can be proved using the properties of similar triangles. As E moves, the triangles ADE and A'DE remain similar, and the ratio of the sides AD/A'